<a href="https://colab.research.google.com/github/bartoszrud/free-form-VQA/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 35.2 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 57.9 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt

from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
import pandas as pd
from pprint import pprint

import sys
import os
import glob

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c quora-question-pairs

403 - Forbidden


In [9]:
!kaggle kernels output abdokamr/question-answering-with-t5 -p /path/to/dest

tcmalloc: large alloc 2675138560 bytes == 0xc3986000 @  0x7fa3920fc1e7 0x4a3940 0x5b438c 0x5b46f7 0x59afff 0x515655 0x59a257 0x570bf0 0x511ee1 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x511e2c 0x549576 0x4bca8a
Output file downloaded to /path/to/dest/lightning_logs/version_0/checkpoints/epoch=2-step=653.ckpt
Output file downloaded to /path/to/dest/lightning_logs/version_0/events.out.tfevents.1628867355.3a1b18457441.24.0
Output file downloaded to /path/to/dest/lightning_logs/version_0/hparams.yaml
Output file downloaded to /path/to/dest/outputs/SimpleT5-epoch-0-train-loss-2.5327/config.json
Output file downloaded to /path/to/dest/outputs/SimpleT5-epoch-0-train-loss-2.5327/pytorch_model.bin
Output file downloaded to /path/to/dest/outputs/SimpleT5-epoch-0-train-loss-2.5327/special_tokens_map.json
Output file downloaded to /path/to/dest/outputs/SimpleT5-epoch-0-

In [10]:
!kaggle datasets download abdokamr/mohler

  0% 0.00/82.7k [00:00<?, ?B/s]
100% 82.7k/82.7k [00:00<00:00, 51.4MB/s]


In [11]:
import zipfile

In [12]:
!unzip mohler.zip

Archive:  mohler.zip
  inflating: mohler_dataset_edited.csv  


In [ ]:
PATH = '/content/gdrive/MyDrive/quora_duplicate_questions.tsv'

In [13]:
dataset = pd.read_csv('/content/mohler_dataset_edited.csv')
dataset.head()

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
0,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,4.0,3.0,3.5
1,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,5.0,5.0
2,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...,5.0,3.0,4.0
3,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,5.0,5.0
4,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is used to let the users have a first idea ...,3.0,3.0,3.0


In [ ]:
dataset = pd.read_csv(PATH, sep="\t")
dataset.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
dataset = dataset.drop(['id', 'qid1', 'qid2', 'is_duplicate'], axis=1)
dataset.head()

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [14]:
!pip install --upgrade simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 42.3 MB/s 
     |████████████████████████████████| 3.5 MB 54.2 MB/s 
     |████████████████████████████████| 527 kB 69.3 MB/s 
     |████████████████████████████████| 140 kB 60.8 MB/s 
     |████████████████████████████████| 829 kB 68.2 MB/s 
     |████████████████████████████████| 409 kB 58.8 MB/s 
     |████████████████████████████████| 952 kB 63.4 MB/s 
     |████████████████████████████████| 880 kB 46.1 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 144 kB 76.1 MB/s 
     |████████████████████████████████| 271 kB 74.3 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=f5dd36ddd77cf5f853133e2589bf5277ad6a59bdfb5e06f126dc96cd627befdb
  Stored in directory: /root/.cache/pip/wheels/30/1b/9

In [ ]:
q_list = "question: " + dataset['question1']                          # questions list to feed the model
n_list = dataset['question2'] + " </s>" # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
df.head()     

,source_text,target_text
0,question: What is the step by step guide to in...,What is the step by step guide to invest in sh...
1,question: What is the story of Kohinoor (Koh-i...,What would happen if the Indian government sto...
2,question: How can I increase the speed of my i...,How can Internet speed be increased by hacking...
3,question: Why am I mentally very lonely? How c...,Find the remainder when [math]23^{24}[/math] i...
4,question: Which one dissolve in water quikly s...,Which fish would survive in salt water? </s>


In [ ]:
!pip install --upgrade simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
q_list = "question: " + dataset['question']                          # questions list to feed the model
n_list = dataset['desired_answer'] + dataset['student_answer'] + " </s>"  # answers list to feed the model

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
df.head() 

,source_text,target_text
0,question: What is the role of a prototype prog...,To simulate the behaviour of portions of the d...
1,question: What is the role of a prototype prog...,To simulate the behaviour of portions of the d...
2,question: What is the role of a prototype prog...,To simulate the behaviour of portions of the d...
3,question: What is the role of a prototype prog...,To simulate the behaviour of portions of the d...
4,question: What is the role of a prototype prog...,To simulate the behaviour of portions of the d...


In [16]:
df['source_text'][0]

'question: What is the role of a prototype program in problem solving?'

In [17]:
from simplet5 import SimpleT5

Global seed set to 42


In [18]:
train_data, val_data = train_test_split(df[:-100], test_size=0.2)
test_data = df[-100:]
train_data.shape, val_data.shape, test_data.shape

((1738, 2), (435, 2), (100, 2))

In [19]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df = train_data,
            eval_df = val_data, 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=10, use_gpu=True)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  copyfile(self.vocab_file, out_vocab_file)


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [20]:
!ls ./outputs

simplet5-epoch-0-train-loss-2.5033-val-loss-1.4704
simplet5-epoch-1-train-loss-1.5252-val-loss-1.2622
simplet5-epoch-2-train-loss-1.2941-val-loss-1.2071
simplet5-epoch-3-train-loss-1.1768-val-loss-1.1827
simplet5-epoch-4-train-loss-1.0755-val-loss-1.1807
simplet5-epoch-5-train-loss-1.0002-val-loss-1.1661
simplet5-epoch-6-train-loss-0.9299-val-loss-1.179
simplet5-epoch-7-train-loss-0.8729-val-loss-1.2134
simplet5-epoch-8-train-loss-0.8106-val-loss-1.2129
simplet5-epoch-9-train-loss-0.7595-val-loss-1.2323


In [21]:
model.load_model("t5","./outputs/simplet5-epoch-9-train-loss-0.7595-val-loss-1.2323")

In [22]:
test_data['source_text']

2173    question: What are the main operations associa...
2174    question: What are the main operations associa...
2175    question: What are the main operations associa...
2176    question: What are the main operations associa...
2177    question: What are the main operations associa...
                              ...                        
2268    question: How many steps does it take to searc...
2269    question: How many steps does it take to searc...
2270    question: How many steps does it take to searc...
2271    question: How many steps does it take to searc...
2272    question: How many steps does it take to searc...
Name: source_text, Length: 100, dtype: object

In [23]:
q_test = test_data['source_text'][2190]
q_ans = test_data['target_text'][2190]

print("Question: ", q_test)
print('-'*50)
print("Answer: ",q_ans)

Question:  question: What is the Euler tour traversal of a tree?
--------------------------------------------------
Answer:  A walk around the tree, starting with the root, where each node is seen three times: from the left, from below, from the right.The Euler tour traversal of a tree begins at the root and visits each node in the tree from the left, then bottom, then right sides. </s>


In [24]:
predicted_ans = model.predict(q_test)[0]

print(predicted_ans)

Traverse the left subtree, then the root, then the right subtree.The Euler tour traversal of a tree takes you from the root to the tip of the tree


In [25]:
test_pd = pd.DataFrame({'question': [q_test], 'answer': [q_ans], 'predicted answer': [predicted_ans]})
test_pd 

,question,answer,predicted answer
0,question: What is the Euler tour traversal of ...,"A walk around the tree, starting with the root...","Traverse the left subtree, then the root, then..."
